In [48]:
import numpy as np
import pandas as pd
import sys
sys.path.append('/Users/kevingmagana/DSI/capstone/capstone-update/Capstone_Proposal/py_scripts/')
import results
from bs4 import BeautifulSoup
%reload_ext autoreload
%autoreload 2
import requests
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer




In [3]:
df = pd.read_csv('/Users/kevingmagana/DSI/capstone/case_corpus.csv')
df = df.dropna()
df['title_date'] = df[['case_title', 'date']].apply(lambda x: ' '.join(x), axis=1)


df2 = df.iloc[:, [2, 12]]
df2 = df2.drop_duplicates(subset='title_date')
df2 = df2.reset_index(drop=True)


In [4]:


model = results.load_model()
def process_query(query):
    """
    :param query: string, the query
    :return: list, of indices or DOC IDs
      EX:
      	 [2939, 4213, 3703, 695, 1810, 3397, 3311, 2663, 177, 1587]
    """
    # Step 1: Intake and get Indexed Cases
    query =  query
    indices = results.indexed_results(model[0], query)
    # Step 2: Get index_vectors and case_val dictionary
    index_vectors, case_value_dictionary= results.get_index_vectors(model[2], indices)
    # Step 3: Vectorize query
    vectorized_query = results.query_vector(model[1], query)
    # Step 4: Get Cosine similarities
    cosine_similarities = results.cosine_similarity(vectorized_query, index_vectors)
    # Step 5: Get top results and rank them (top 10 right now...)
    top_results = (results.get_top_values(cosine_similarities[0], 10), query)
    res = [case_value_dictionary[array_value] for array_value in top_results[0]]
    return res


In [27]:
print(model[3].case_text[4213])

United States Court of Appeals,Ninth Circuit.
Jesus Aaron CAZAREZ-GUTIERREZ, Petitioner, v. John ASHCROFT, Attorney General, Respondent.
No. 02-72978.
Decided: August 24, 2004

Before:  HUG, B. FLETCHER, and TASHIMA, Circuit Judges.Jose A. Bracamonte, Law Office of Jose A. Bracamonte, Phoenix, AZ, for the petitioner-appellant. Anthony Payne, U.S. Department of Justice, Civil Division, Washington, DC, for the respondent-appellee.
Jesus Aaron Cazarez-Gutierrez (“Cazarez-Gutierrez” or Petitioner) petitions for review of a decision of the Board of Immigration Appeals (BIA or “Board”) finding him statutorily ineligible for cancellation of removal because he was convicted of the “aggravated felony” of a “drug trafficking crime.”   Cazarez-Gutierrez argues that his state felony conviction for possession of methamphetamine, which would be a misdemeanor if prosecuted under federal law, should not be classified as an aggravated felony for immigration purposes.I. BACKGROUNDCazarez-Gutierrez is a 

In [8]:
html = model[3].html_format[4213]

# content = requests.get(html).content
soup = BeautifulSoup(html, 'lxml')


list_of_paragraphs = []
for paragaph in soup.findAll('p'): 
    list_of_paragraphs.append(str(paragaph.text.encode('ascii', 'ignore')).replace("'", ""))
# print(model[3].html_format[0])

len(list_of_paragraphs)

76

In [11]:
### Step 1: Vectorize and apply Tf-IDF weights on the case

cleaned_text = clean_text(list_of_paragraphs)

# tokenized= tokenize(list_of_paragraphs)
# remove_stops = remove_stop_words(tokenized)
# cleaned_text = stem_words(remove_stops)

list_of_strings = [" ".join(lst) for lst in cleaned_text]

vectorizer = TfidfVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(list_of_strings)

vectors.shape, len(list_of_paragraphs)

((76, 1122), 76)

In [12]:
query = ['What is the penalty for drug trafficking?']
### Step 2: Vectorize the query

cleaned_query = clean_text(query)
query_list_of_strings = [" ".join(lst) for lst in cleaned_query]
tokenized_query = vectorizer.transform(query_list_of_strings)

cosine_similarities = linear_kernel(tokenized_query, vectors) 
top_results = (get_top_values(cosine_similarities[0], 10), query)
print(top_results)

([69, 20, 44, 40, 39, 14, 37, 46, 51, 68], ['What is the penalty for drug trafficking?'])


In [13]:
top_results[0]

[69, 20, 44, 40, 39, 14, 37, 46, 51, 68]

In [14]:
# print(tokenized_query.todense())
# vectorizer.vocabulary_
# print(list_of_strings)
# data= model[3].case_text[2939].split('\n')
print(query)
# print(list_of_strings)
print(list_of_paragraphs[69])


['What is the penalty for drug trafficking?']
b5.As noted above, aggravated felony includes the illicit trafficking in a controlled substance  including a drug trafficking crime (as defined in section 924(c) of Title 18).  Although it does not matter in application, we note that a drug offense with a trafficking element is illicit trafficking and a drug offense that is punishable as a felony under federal law is a drug trafficking crime.


In [15]:
def highlight_key_query_words(question, most_relevant_part):
    """
    :param question:
    :param most_relevant_part:
    :return:
    """
    # print(question)
    # print(most_relevant_part)
    final_results = []
    stop = set(stopwords.words('english'))
    parse_query = "".join(question).split(' ')
    parse_query = clean_text(parse_query)
    
    most_relevant = most_relevant_part.split(' ')
    check_result = clean_text(most_relevant)
    # print(len(most_relevant))
    # print(len(check_result))
    #
    # print(most_relevant)
    # print(check_result)

    for ind, word in enumerate(most_relevant):
        if check_result[ind] in parse_query and check_result[ind] != []:
#             final_results.append('<b>' + word + '</b>') ## If rendering on HTML
            final_results.append('\033[1m' + word + '\033[0m') ## Python
        else:
            final_results.append(word)

#     return " ".join(('...', ' '.join(final_results), '<b>' + '... Continue reading case' + '</b>' )) HTML
    return " ".join(('...', ' '.join(final_results), '\033[1m' + '... Continue reading case' + '\033[0m')) ## Python

def get_most_relevant_part(case_index, corpus_df, query, top_n=3, highlight_num=1):
    """
    :param case_index: int, the index of the case
    :param corpus_df: pandas dataframe, for the entire corpus of cases
    :param query: string, of the web-input query
    :param top_n: int, the top n matching paragraphs with query
    :returns query: the original query,
            top_result: list, of top indexed matching paragraphs,
            list_of_paragraphs:

    Two use cases: 
        (1) if highlight_num == 1 then it's to return for single highlighting. 
            See highlight_key_query_words() for example. 
        (2) if highlight_num > 1 then this function is for returning the list of 3 most
        relevant parts of a text to be highlighted once a user clicks on the search result. 
    
    """
    ## Step 1: Load up the data frame
    html = corpus_df.html_format[case_index]
    soup = BeautifulSoup(html, 'lxml')

    ## Step 2: Parse the case into a list of strings
    list_of_paragraphs = []
    for paragaph in soup.findAll('p'):
        list_of_paragraphs.append(paragaph.getText())

    ## Step 3: Parse case and vectorize with Tf-IDF weights
    cleaned_text = clean_text(list_of_paragraphs)
    list_of_strings = [" ".join(lst) for lst in cleaned_text]
    vectorizer = TfidfVectorizer(stop_words='english')
    vectors = vectorizer.fit_transform(list_of_strings)

    ## Step 4: Vectorize Query
#     query_list = [query]
    cleaned_query = clean_text(query)
    query_list_of_strings = [" ".join(lst) for lst in cleaned_query]
    tokenized_query = vectorizer.transform(query_list_of_strings)

    ## Step 5: Run Cosine Similarity on Query and Case Text
    cosine_similarities = linear_kernel(tokenized_query, vectors)
    top_results = [i for i in np.argsort(cosine_similarities[0])[-1:-top_n - 1:-1]]

    if highlight_num == 1: 
        highlight_output = list_of_paragraphs[top_results[0]]
    
    elif highlight_num == 2: 
        highlight_output = [list_of_paragraphs[ind] for ind in top_results[0:2]]  
    else: 
        highlight_output = [list_of_paragraphs[ind] for ind in top_results[0:3]]   
        
    return highlight_output  ## Returns top 3 indexed cases


## QUERY: What is the penalty for drug trafficking?
## Index of RESULTS [2939, 4213, 3703, 695, 1810, 3397, 3311, 2663, 177, 1587]


In [50]:
# What is the penalty for drug trafficking?
## FLASK RESULT: [2269, 1219, 1416, 2458, 4468, 1769, 1948, 2035, 4466, 1927]
## result.py RESULT: [1537, 1659, 2446, 1761, 1868, 1927, 82, 1272, 864, 3177]




In [51]:

## Make sure query is in a LIST, otherwise WON'T WORK!
legal_query = ['What is the penalty for drug trafficking?']
data = model[3]
index = 4213
html = data.html_format[index]

results = get_most_relevant_part(index, data, legal_query, top_n=3, highlight_num = 1)
# print(results)
highlights = highlight_key_query_words(legal_query, results)
print(highlights)

# print(list_of_paragraphs[23])


... 5.   As noted above, “aggravated felony” includes the “illicit trafficking in a controlled substance ․ including a drug trafficking crime (as defined in section 924(c) of Title 18).”   Although it does not matter in application, we note that a drug offense with a trafficking element is “illicit trafficking” and a drug offense that is punishable as a felony under federal law is a “drug trafficking crime.” ... Continue reading case


## Finding top 3 most relevant parts of the text

In [57]:

results = get_most_relevant_part(index, data, legal_query, top_n=3, highlight_num = 3)

# print(len(results))

list_of_top_3 = [res for res in results]
string_of_top_3 = str(" ".join(list_of_top_3).encode("ascii", "ignore"))
print(string_of_top_3)

b'5.As noted above, aggravated felony includes the illicit trafficking in a controlled substance  including a drug trafficking crime (as defined in section 924(c) of Title 18).  Although it does not matter in application, we note that a drug offense with a trafficking element is illicit trafficking and a drug offense that is punishable as a felony under federal law is a drug trafficking crime. The Hernandez-Avalos Court reached its conclusion by following cases interpreting the statutory definition of drug trafficking crime in the context of sentencing enhancement. 251 F.3d at 508 (citing United States v. Hinojosa-Lopez, 130 F.3d 691 (5th Cir.1997)).  Under the Sentencing Guidelines, anyone convicted of unlawful reentry after removal is subject to an enhanced penalty if she previously was convicted of an aggravated felony in the United States. U.S.S.G. 2L1.2. The Sentencing Guidelines previously defined aggravated felony with reference to the INA, 8 U.S.C. 1101(a)(43), which in turn in

In [24]:
html = data.html_format[index]

# content = requests.get(html).content
soup = BeautifulSoup(html, 'lxml')

list_of_paragraphs = []
for paragaph in soup.findAll('p'): 
    list_of_paragraphs.append(paragaph.getText())
# print(model[3].html_format[0])

print(len(list_of_paragraphs))
print(list_of_paragraphs[69])



76
5.   As noted above, “aggravated felony” includes the “illicit trafficking in a controlled substance ․ including a drug trafficking crime (as defined in section 924(c) of Title 18).”   Although it does not matter in application, we note that a drug offense with a trafficking element is “illicit trafficking” and a drug offense that is punishable as a felony under federal law is a “drug trafficking crime.”


In [64]:
# print(data.html_format[index])

# raw_text = [word.getText() for word in soup.prettify()]  

# list_of_paragraphs
# str_= str(html.encode("ascii", "ignore").replace("\n", ''))
# print(str_)

data.head()

,Unnamed: 0,Unnamed: 0.1,case_text,case_title,court,date,docket,tags,type_of_law,url,web_source,html_format,title_date
0,0,1,"United States Court of Appeals,Tenth Circuit.\...",Porro v. Barnes,United States Tenth Circuit,11/09/2010,10-6002,"Civil Rights, Constitutional Law, Immigration ...",case,http://caselaw.findlaw.com/us-10th-circuit/154...,findlaw.com,"[<div class=""caselawcontent searchable-content...",Porro v. Barnes 11/09/2010
1,1,2,"United States Court of Appeals,Tenth Circuit.\...",Garcia-Carbajal v. Holder,United States Tenth Circuit,11/05/2010,09-9558,"Administrative Law, Immigration Law",case,http://caselaw.findlaw.com/us-10th-circuit/154...,findlaw.com,"[<div class=""caselawcontent searchable-content...",Garcia-Carbajal v. Holder 11/05/2010
2,2,3,"United States Court of Appeals,Tenth Circuit.\...",US v. Rendon-Alamo,United States Tenth Circuit,10/19/2010,10-2089,"Sentencing, Immigration Law, Criminal Law & Pr...",case,http://caselaw.findlaw.com/us-10th-circuit/154...,findlaw.com,"[<div class=""caselawcontent searchable-content...",US v. Rendon-Alamo 10/19/2010
3,3,5,"United States Court of Appeals,Tenth Circuit.\...",Iliev v. Holder,United States Tenth Circuit,07/19/2010,09-9517,"Administrative Law, Immigration Law",case,http://caselaw.findlaw.com/us-10th-circuit/153...,findlaw.com,"[<div class=""caselawcontent searchable-content...",Iliev v. Holder 07/19/2010
4,4,6,"United States Court of Appeals,Tenth Circuit.\...",US v. Adame-Orozco,United States Tenth Circuit,06/04/2010,09-3296,"Criminal Law & Procedure, Immigration Law",case,http://caselaw.findlaw.com/us-10th-circuit/152...,findlaw.com,"[<div class=""caselawcontent searchable-content...",US v. Adame-Orozco 06/04/2010


## Sentimentality Scoring

In [43]:
from textblob import TextBlob
# document = model[3].case_text[4213]
# string = string()
stringy = str(" ".join(list_of_paragraphs).encode("ascii", "ignore"))

opinion = TextBlob(string)
print (opinion.sentiment)

from textblob.sentiments import NaiveBayesAnalyzer
opinion = TextBlob(string, analyzer=NaiveBayesAnalyzer())
print (opinion.sentiment)


Sentiment(polarity=0.02262087090606946, subjectivity=0.4221132157052735)
Sentiment(classification='pos', p_pos=1.0, p_neg=8.860405334569313e-85)


###  Looks like the WHOLE document is classified as low polarity, and lesss subject (more objective)

### Now let's look at just the TOP 3 highlighted texts

In [60]:
####QUERY: What is the penalty for drug trafficking?
legal_query = ['What is the penalty for drug trafficking?']
data = model[3]
index = 4213
html = data.html_format[index]
results = get_most_relevant_part(index, data, legal_query, top_n=3, highlight_num = 3)

list_of_top_3 = [res for res in results]
string_of_top_3 = str(" ".join(list_of_top_3).encode("ascii", "ignore"))

opinion = TextBlob(string_of_top_3)
print (opinion.sentiment[0])

opinion = TextBlob(string_of_top_3, analyzer=NaiveBayesAnalyzer())
print (opinion.sentiment)

-0.056862745098039215
Sentiment(classification='pos', p_pos=0.9999999729376016, p_neg=2.7062417380635343e-08)


## Looks like in the most relevant portions of the case -- there's a negative polarity, meaning a negative sentiment overall. Whereas the classifier is a positive classification... 

## There's also the differenc between 

In [62]:
OldMax, OldMin = 1, -1
NewMax, NewMin = 10, 0
OldValue = -0.056862745098039215

OldRange = (OldMax - OldMin)  
NewRange = (NewMax - NewMin)  
NewValue = (((OldValue - OldMin) * NewRange) / OldRange) + NewMin

print(NewValue)

4.715686274509804


In [10]:
## QUERY: What is the penalty for drug trafficking?
## RESULTS [2939, 4213, 3703, 695, 1810, 3397, 3311, 2663, 177, 1587]

def get_top_values(lst, n):
    '''
    INPUT: LIST, INTEGER, LIST
    OUTPUT: LIST

    Given a list of values, find the indices with the highest n values.
    Return the labels for each of these indices.

    e.g.
    lst = [7, 3, 2, 4, 1]
    n = 2
    labels = ["cat", "dog", "mouse", "pig", "rabbit"]
    output: ["cat", "pig"]
    
    [-1:-n-1:-1]
    '''
    return [i for i in np.argsort(lst)[-1:-n-1:-1]]

def most_relevant_part_of_one_text(case, query): 
    
    
    vectorizer = TfidfVectorizer(stop_words='english')
    vectors = vectorizer.fit_transform(case).toarray()
        
    tokenized_query = vectorizer.transform(query)
    cosine_similarities = results.cosine_similarity(vectorized_query, index_vectors)
    
    top_results = (results.get_top_values(cosine_similarities[0], 10), query)

    return top_results


def tokenize(tokenize_texts):
    """
    :param case_text: list of strings (case_texts)
    :returns: list of strings, tokenized word.
    """
    table = str.maketrans({key: None for key in string.punctuation})
    return [word_tokenize(case.translate(table).lower()) for case in tokenize_texts]

def remove_stop_words(remove_stop_text):
    """
    :param case_text: list of lists of strings (tokens)
    :returns: list of lists of strings(tokens) without stopwords
    """
    stop = set(stopwords.words('english'))
    return [[word for word in words if word not in stop] for words in remove_stop_text]

def stem_words(stem_word_text):
    """
    :param without_stop: list of lists without stop words
    :return: list of lists of strings (stemmed words)
    """
    snowball = SnowballStemmer('english')
    return [[snowball.stem(word) for word in words] for words in stem_word_text]


def clean_text(case_text):
    """
    :param case_text: list of strings (case_text)
    :return: list of lists of strings (stemmed words)
    """
    
    table = str.maketrans({key: None for key in string.punctuation})
    tokenized = [word_tokenize(case.translate(table).lower()) for case in case_text]
    stop = set(stopwords.words('english'))
    remove_stop_words = [[word for word in words if word not in stop] for words in tokenized]
    snowball = SnowballStemmer('english')
    ## Stemmed words 
    return [[snowball.stem(word) for word in words] for words in remove_stop_words]

# most_relevant_part_of_one_text(model[3].case_text[0], query)

# ## Step 

# vectorizer = TfidfVectorizer(stop_words='english')
# vectors = vectorizer.fit_transform(data).toarray()
